In [1]:
import joblib
import numpy as np
import pandas as pd

# 1. Carregar os artefatos
scaler = joblib.load('03_Modelos_Salvos/scaler_padronizador.pkl')
modelo = joblib.load('03_Modelos_Salvos/modelo_final.pkl')
dados_treino = joblib.load('01_Base_Dados/dados_finalizados.pkl')

print("Artefatos carregados com sucesso!")

Artefatos carregados com sucesso!


In [2]:
# 2. Carregar o CSV original apenas para pegar um exemplo real (não escalonado)
df_original = pd.read_csv('01_Base_Dados/AmesHousing.csv')

# 3. Função para facilitar a previsão
def prever_valor_imovel(dados_casa_df):
    """
    Recebe um DataFrame com as mesmas colunas do treino, aplica o scaler e retorna o valor em USD.
    """
    
    dados_casa_df = dados_casa_df[dados_treino['features']]
    
    # Aplica o Scaler
    dados_escalonados = scaler.transform(dados_casa_df)
    
    # Predição e Reversão do Log para Dólares
    pred_log = modelo.predict(dados_escalonados)
    return np.expm1(pred_log)[0]

A variável alvo (SalePrice) possui uma distribuição assimétrica para a direita (right-skewed).
- Apliquei a transformação logarítmica antes de reverter para dólares pois:
1. Normalização: Aproxima a distribuição da variável alvo de uma distribuição normal.
2. Estabilização da Variância: Reduz o impacto de preços muito elevados (outliers), evitando que o modelo priorize o acerto em casas de luxo em detrimento das casas populares.
3. Penalização Proporcional: No espaço logarítmico, o erro é tratado de forma percentual. 
* Errar $10k em uma casa de $100k passa a ter o mesmo peso que errar $100k em uma casa de $1M.

In [3]:
# --- TESTE COM UMA CASA REAL DO DATASET ---

# Primeira casa do dataset original que não foi usada no treino:
indice_exemplo = 10 
casa_real_bruta = df_original.iloc[[indice_exemplo]].copy()

casa_exemplo_processada = pd.get_dummies(casa_real_bruta).reindex(columns=dados_treino['features'], fill_value=0)

# Executar Predição
valor_real_venda = df_original.iloc[indice_exemplo]['SalePrice']
valor_predito = prever_valor_imovel(casa_exemplo_processada)

print("--- 🏠 RELATÓRIO DE AVALIAÇÃO REAL ---")
print(f"Valor Real no Catálogo: ${valor_real_venda:,.2f}")
print(f"Valor Estimado pelo Modelo: ${valor_predito:,.2f}")
print(f"Diferença Absoluta: ${abs(valor_real_venda - valor_predito):,.2f}")
print(f"Precisão: {100 - (abs(valor_real_venda - valor_predito)/valor_real_venda*100):.2f}%")

--- 🏠 RELATÓRIO DE AVALIAÇÃO REAL ---
Valor Real no Catálogo: $175,900.00
Valor Estimado pelo Modelo: $143,605.97
Diferença Absoluta: $32,294.03
Precisão: 81.64%
